In [1]:
import pandas as pd
import numpy as np
import csv
import ast
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
import time
import logging
logging.getLogger().setLevel(logging.INFO)
logging.basicConfig(
    filename='scraping_missing.log',
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [49]:
scraped_details_df = pd.read_csv('aryaman/csv/scraping/scraping_details.csv',encoding='utf-8')
products = pd.read_csv('aryaman/csv/categories_and_products/products.csv')
missing_details_df = pd.read_csv("aryaman/csv/scraping/MissingDetailsChart.csv")

In [3]:
scraped_details_df['Images'] = scraped_details_df['Images'].apply(lambda item: ast.literal_eval(item) if pd.notna(item) else item)
scraped_details_df['productOverview'] = scraped_details_df['productOverview'].apply(lambda item: ast.literal_eval(item) if pd.notna(item) else item)
scraped_details_df['featureBullets'] = scraped_details_df['featureBullets'].apply(lambda item: ast.literal_eval(item) if pd.notna(item) else item)

In [6]:
bullets_and_overview_same_df = missing_details_df[missing_details_df['featureBullets']== missing_details_df['Overview']]
diff_bullets_and_overview_df = pd.concat([bullets_and_overview_same_df,missing_details_df]).drop_duplicates(keep=False)
diff_bullets_and_overview_df[['featureBullets','Overview']].value_counts()

featureBullets  Overview
False           True        259
True            False        67
Name: count, dtype: int64

In [8]:
merged_df = pd.merge(products,missing_details_df[(missing_details_df['featureBullets']==True) & (missing_details_df['Overview']==True)],on='ASIN',how='inner')

In [19]:
webdriver_path = "..\chromedriver.exe"
cService = webdriver.ChromeService(executable_path=webdriver_path)
# chrome_options = Options()
# chrome_options.add_experimental_option("detach",True)
driver = webdriver.Chrome(service=cService)

In [16]:
# with open('scraping_missing_details_featureBullets_Overview.csv','w',newline='',encoding='utf-8') as file:
#     writer = csv.DictWriter(file,fieldnames=['ASIN',  'productOverview', 'featureBullets'])
#     writer.writeheader()

In [17]:
# pd.DataFrame({"blacklisted_asins":[]}).to_csv('blacklisted2.csv',index=False)

In [18]:
def filter_unscraped_iter(products_links_df,scraped_details_df):
    return list(products_links_df[~(products_links_df['ASIN'].isin(scraped_details_df['ASIN']))].itertuples(index=False))

In [77]:
scraped_missing_details_df = pd.read_csv('scraping_missing_details_featureBullets_Overview.csv',encoding='utf-8')
products_links = merged_df[['ASIN','Product Link']]
products_links.reset_index(inplace=True,drop=True)
blacklisted_asins = (pd.read_csv('blacklisted2.csv'))['blacklisted_asins'].tolist()
missing_details_update_df = pd.read_csv("MissingDetailsUpdated.csv")
no_of_items_to_scrape = 1000
how_many_scraped = 0
with open('scraping_missing_details_featureBullets_Overview.csv','a',newline='',encoding='utf-8') as file:
    writer = csv.DictWriter(file,fieldnames=['ASIN','productOverview','featureBullets'])
    iterator_to_loop = filter_unscraped_iter(products_links,scraped_missing_details_df)
    for link in iterator_to_loop:
        if link.ASIN in blacklisted_asins:
            continue
        driver.get(link._1)
        time.sleep(1)
        try:
            productFactsDiv = driver.find_element(By.ID,'productFactsDesktop_feature_div')
            try:
                overview = dict()
                headings = productFactsDiv.find_elements(By.TAG_NAME,'h3')
                product_details_heading = None
                for heading in headings:
                    if ((driver.execute_script("return arguments[0].textContent",heading)).strip() == 'Product details'):
                        product_details_heading = heading
                if product_details_heading != None:
                    missing_details_update_df.loc[missing_details_update_df['ASIN']==link.ASIN,'Overview'] =  False
                    logging.info(f"missing details updated {link.ASIN}")
                    nextSibling = driver.execute_script("return arguments[0].nextElementSibling",product_details_heading)
                    while( nextSibling.tag_name == 'div' and 'product-facts-detail' in nextSibling.get_attribute("class")):
                        spans = nextSibling.find_elements(By.CSS_SELECTOR,"span.a-color-base")
                        if len(spans) == 2:
                            if len(spans[0].find_elements(By.CSS_SELECTOR,"*")) == len(spans[1].find_elements(By.CSS_SELECTOR,"*")) == 0:
                                overview[driver.execute_script("return arguments[0].textContent",spans[0])] = driver.execute_script("return arguments[0].textContent",spans[1])
                        nextSibling = driver.execute_script("return arguments[0].nextElementSibling",nextSibling)
            except NoSuchElementException:
                print("over view not found",link.ASIN)
                logging.info(f"overview not found {link.ASIN}")
            except Exception as e:
                print(f"Exception in headings {link.ASIN}",e)
                break
            try:
                bullets_text = list()
                list_of_bullets = productFactsDiv.find_element(By.TAG_NAME,'ul')
                bullets = list_of_bullets.find_elements(By.TAG_NAME,'li')
                if len(bullets) == 0:
                    raise ValueError
                missing_details_update_df.loc[missing_details_update_df['ASIN']==link.ASIN,'featureBullets'] =  False
                logging.info(f"missing details updated {link.ASIN}")
                for bullet in bullets:
                    spans = bullet.find_elements(By.TAG_NAME,'span')
                    if len(spans) == 1:
                        if len(spans[0].find_elements(By.CSS_SELECTOR,"*")) == 0:
                            bullets_text.append((driver.execute_script("return arguments[0].textContent",spans[0])))
                
            except NoSuchElementException:
                print("bullets not found ",link.ASIN)
                logging.info(f"bullets not found {link.ASIN}")
            except Exception as e:
                print(f"Exception in bullets {link.ASIN}",e)
                break
            if len(overview)>0 or len(bullets_text) > 0:
                if len(overview) == 0:
                    overview = None
                if len(bullets_text) == 0:
                    bullets_text = None
                new_entry = {'ASIN':link.ASIN,'productOverview':overview,'featureBullets':bullets_text}
                writer.writerow(new_entry)
                file.flush()
            missing_details_update_df.to_csv('MissingDetailsUpdated.csv',index=False)
            no_of_items_to_scrape -= 1
            if no_of_items_to_scrape == 0:
                print('break normally')
                break
            how_many_scraped += 1
            if how_many_scraped == 50:
                print(1000-no_of_items_to_scrape)
                how_many_scraped = 0
            
        except ValueError as e:
            print(f"value error occurred,breaking {link.ASIN}",e)
            break
        except NoSuchElementException:
            # print("Blacklisting")
            blacklisted_asins.append(link.ASIN)
            pd.DataFrame({'blacklisted_asins':blacklisted_asins}).to_csv('blacklisted2.csv',index=False)
            logging.warning(f"blacklisted - {link.ASIN}")
        except Exception as e:
            print(f"some exception occurred,breaking {link.ASIN}",e)
            break
missing_details_update_df.to_csv('MissingDetailsUpdated.csv',index=False)
logging.info(f"missing details flushed to file")

In [78]:
scraped_missing_details_df = pd.read_csv('scraping_missing_details_featureBullets_Overview.csv',encoding='utf-8')

In [44]:
missing_details_update_df = pd.read_csv("MissingDetailsUpdated.csv")

In [109]:
missing_details_update_df[missing_details_update_df['ASIN'].isin(scraped_missing_details_df[scraped_missing_details_df['featureBullets'].isnull()]['ASIN'])]

,ASIN,star_rating,ratings,featureBullets,Overview
2226,B0BQ4ZD1HR,False,False,True,False
2264,B08D3L7853,False,False,True,False
2334,B09NN4WSD8,False,False,True,False
2442,B08FZS2NFS,False,False,True,False
2936,B08D3PQ8Z6,False,False,True,False


In [110]:
blacklisted_asins_df = pd.read_csv('blacklisted2.csv')

In [123]:
missing_details_update_df[missing_details_update_df['ASIN'].isin(blacklisted_asins_df['blacklisted_asins'])].reset_index(drop=True).equals(missing_details_df[missing_details_df['ASIN'].isin(blacklisted_asins_df['blacklisted_asins'])].reset_index(drop=True))

True

In [129]:
asins_of_only_bullets = missing_details_df[(missing_details_df['featureBullets']==True) & (missing_details_df['Overview']==False)]['ASIN'].tolist()

In [131]:
# missing_details_df[(missing_details_df['featureBullets']==False) & (missing_details_df['Overview']==True)]
asins_of_only_overview = missing_details_df[(missing_details_df['featureBullets']==False) & (missing_details_df['Overview']==True)]['ASIN'].tolist()

In [135]:
np.random.choice(asins_of_only_overview,5)

array(['B0CQ6HNDZ4', 'B0CY5HVDS2', 'B0BQC6R7B7', 'B07XXFB9CZ',
       'B0D4L8QSWP'], dtype='<U10')

In [120]:
missing_details_df[missing_details_df]

,ASIN,productOverview,featureBullets
1186,B0BQ4ZD1HR,"{'Case diameter': '40 Millimetres', 'Band colo...",NaN
1224,B08D3L7853,"{'Material type': 'Synthetic', 'Closure type':...",NaN
1293,B09NN4WSD8,"{'Material type': 'Mesh', 'Closure type': 'Lac...",NaN
1401,B08FZS2NFS,"{'Material type': 'Mesh', 'Closure type': 'Lac...",NaN
1712,B08D3PQ8Z6,"{'Material type': 'Synthetic', 'Closure type':...",NaN


In [136]:
missing_details_df[missing_details_df['ASIN']=='B0C27TZ53R']

,ASIN,star_rating,ratings,featureBullets,Overview
2717,B0C27TZ53R,False,False,True,True


In [145]:
joined1 = pd.merge(scraped_details_df,missing_details_df,how='inner',on='ASIN')

In [158]:
type(joined1['featureBullets_x'][123])

str

In [152]:
joined1.columns

Index(['ASIN', 'Images', 'title', 'productOverview', 'featureBullets_x',
       'stars', 'ratings_x', 'listPrice', 'salePrice', 'star_rating',
       'ratings_y', 'featureBullets_y', 'Overview'],
      dtype='object')

In [160]:
for index,row in joined1.iterrows():
    if pd.isna(row['featureBullets_x']) != row['featureBullets_y']:
        print(row['ASIN'])
    if pd.isna(row['productOverview']) != row['Overview']:
        print(row['ASIN'])

In [162]:
joined1[joined1['ASIN']=='B0CQFZZVG6']

,ASIN,Images,title,productOverview,featureBullets_x,stars,ratings_x,listPrice,salePrice,star_rating,ratings_y,featureBullets_y,Overview
2626,B0CQFZZVG6,['https://m.media-amazon.com/images/I/61B9xDfv...,Ravaiyaa - Attitude is everything Women Fashio...,NaN,NaN,5.0,1.0,1699.0,499.0,False,False,True,True


In [167]:
x = driver.find_element(By.ID,'bookDescription_feature_div')

In [171]:
y = x.find_elements(By.CSS_SELECTOR,'div.a-expander-content')

In [ ]:
print(driver.execute_script("return arguments[0].textContent",y[0]))

In [2]:
# pd.DataFrame({"ASIN":[],"bookDescription":[]}).to_csv('MostlyBooksRescraped.csv',index=False)

In [5]:
blacklisted_asins = pd.read_csv('blacklisted2.csv')

In [31]:
with open('MostlyBooksRescraped.csv',mode='a',newline='',encoding='utf-8') as file:
    writer = csv.DictWriter(file,fieldnames=['ASIN','bookDescription'])
    # rejected_asins = []
    for i in blacklisted_asins['blacklisted_asins']:
        if i  in mostly_rescraped_books_df['ASIN'].tolist():
            continue
        if i in rejected_asins:
            continue
        link = f"https://www.amazon.in/dp/{i}"
        driver.get(link)
        time.sleep(2)
        try:
            bookDescriptiondiv = driver.find_element(By.ID,'bookDescription_feature_div')
            x = bookDescriptiondiv.find_elements(By.CSS_SELECTOR,'div.a-expander-content')
            if len(x)==1:
                new_entry = dict()
                new_entry['ASIN'] = i
                new_entry['bookDescription'] = driver.execute_script("return arguments[0].innerText",x[0])
                writer.writerow(new_entry)
                file.flush()
            else:
                raise ValueError
        except NoSuchElementException:
            print("NO such element ")
            rejected_asins.append(i)
        except ValueError as e:
            print(f"VAlue eeorr {i} ",e)
            rejected_asins.append(i)
        except Exception as e:
            print(f"exception {i}",e)
            break


In [67]:
mostly_rescraped_books_df = pd.read_csv('MostlyBooksRescraped.csv')
mostly_rescraped_books_df.drop_duplicates(inplace=True)

In [112]:
scraped_missing_details_df = pd.read_csv('scraping_missing_details_featureBullets_Overview.csv',encoding='utf-8')

In [114]:
scraped_missing_details_df.isnull().sum()

ASIN                0
productOverview    16
featureBullets      5
dtype: int64

In [119]:
concatenated_df = pd.concat([scraped_missing_details_df,mostly_rescraped_books_df],axis=0)

In [120]:
concatenated_df.reset_index(drop=True,inplace=True)

In [121]:
concatenated_df

,ASIN,productOverview,featureBullets,bookDescription
0,B07MLLXYKV,"{'Material composition': '100% Polyester', 'Cl...","['Material: 100% Cotton', 'Package Includes: 1...",NaN
1,B08HHRBVVP,"{'Closure type': 'Elastic', 'Care instructions...","['Material: 100% Cotton', 'Package Includes: 1...",NaN
2,B08HWZK311,"{'Material type': 'Polycotton', 'Fit type': 'R...","['Material: 95% Cotton, 5% Polyester', 'Packag...",NaN
3,B0DGTN6JWB,"{'Style': 'Pack of 1', 'Fit type': 'Regular', ...","[""✅ 𝟏𝟎𝟎% 𝑶𝒓𝒈𝒂𝒏𝒊𝒄 𝑪𝒐𝒕𝒕𝒐𝒏: Crafted with the soft...",NaN
4,B08Q2V393N,"{'Material composition': '100% Cotton', 'Patte...",['100% PURE COTTON: BABY GO Set of 3 T-Shirts ...,NaN
...,...,...,...,...
2558,B08GLKNX2V,NaN,NaN,"about the book,\n\ni know i don’t know, is a j..."
2559,8170286913,NaN,NaN,A Treasury of Urdu Poetry is a treasure-house ...
2560,1471165825,NaN,NaN,The stunning new poetry collection from Rupi K...
2561,9357765476,NaN,NaN,About the Book\n\nThis book is a collection of...


In [123]:
concatenated_df.to_csv('concatenated.csv',encoding='utf-8',index=False)

In [104]:
type(concatenated_df[concatenated_df['ASIN'].isin(mostly_rescraped_books_df['ASIN'])]['featureBullets'].iloc[0])

str

In [78]:
scraped_details_df[scraped_details_df['ASIN'].isin(mostly_rescraped_books_df['ASIN'])]

,ASIN,Images,title,productOverview,featureBullets,stars,ratings,listPrice,salePrice
1603,B09X5WGSVG,['https://m.media-amazon.com/images/I/71swBOdh...,INTIMIFY Lavender Essential Oil for Hair Growt...,NaN,NaN,4.3,290.0,599.0,197.0
1701,0143453580,['https://m.media-amazon.com/images/I/71dNsRuY...,YOU ONLY LIVE ONCE,NaN,NaN,4.4,4218.0,250.0,178.0
1702,014345367X,['https://m.media-amazon.com/images/I/61Wwj-Ij...,Where the Sun Never Sets (from the bestselling...,NaN,NaN,4.2,401.0,250.0,212.0
1703,0143453688,['https://m.media-amazon.com/images/I/71nlXl+u...,On The Open Road: Three Lives. Five Citi: Thre...,NaN,NaN,4.3,644.0,250.0,188.0
1704,9362619342,['https://m.media-amazon.com/images/I/51bupq9K...,Struggler,NaN,NaN,NaN,NaN,NaN,211.0
...,...,...,...,...,...,...,...,...,...
2349,B08GLKNX2V,['https://m.media-amazon.com/images/I/21muvS+4...,Poetry book I Know I Don't Know,NaN,NaN,4.6,130.0,319.0,239.0
2350,8170286913,['https://m.media-amazon.com/images/I/412Vzql1...,A Treasury of Urdu Poetry,NaN,NaN,4.1,126.0,495.0,364.0
2351,1471165825,['https://m.media-amazon.com/images/I/41kbEWXM...,The Sun and her flowers,NaN,NaN,4.5,26357.0,1390.0,357.0
2352,9357765476,['https://m.media-amazon.com/images/I/41FAdLiu...,Lucid Road,NaN,NaN,5.0,5.0,NaN,599.0


NameError: name 'f' is not defined

In [48]:
missing_details_update_df[missing_details_update_df['ASIN'].isin(mostly_rescraped_books_df['ASIN'])]['Overview'].all()

np.True_

In [109]:
mostly_rescraped_books_df.drop(index=mostly_rescraped_books_df[mostly_rescraped_books_df['ASIN']=='B09X5WGSVG'].index,inplace=True)

In [110]:
df2 = products[products['ASIN'].isin(mostly_rescraped_books_df['ASIN'])]

In [111]:
df2['Main Category'].unique()

array(['Books'], dtype=object)

In [61]:
df2[df2['Main Category']=='Beauty & Personal Care']

,Main Category,Tag,ASIN,Product Link
1677,Beauty & Personal Care,Essential Oils,B09X5WGSVG,https://www.amazon.in/dp/B09X5WGSVG


'B0CXK72DYT',
 'B0D13KTZ53',
 'B0CGRD1BBM',
 'B0DCZZ62PF',
 'B0BVVHLBR4',
 'B0CVQGF48R',
 'B0D6BCVJQ9',
 'B0CKZC8R8C',
 'B0B4KMZQV9',
 'B0D6RTL478',
 'B0D6RV5L9K',
 'B0D6RTT1PR',
 'B0BDMMBR89',
 'B0BWYV7BKL',
 'B0CVQMVDN6',
 'B0CVQNYJ5V',
 'B0BZWLT87W',
 'B0D6RS9L73',
 'B0CL9CVXXB',
 'B0CVQRK9B8',
 'B0C5X4GRS7',
 'B0B3TMLPMJ',
 'B07JMDN7CZ',
 '9384225487',
 'B0CQFZZVG6',
 'B0CQG4XY26',
 'B0C27TZ53R',
 'B0DCK4QTR5',
  'B0B7WPY1HC',
 'B0C8BQNTHK',
 'B0992GZF38']